<a href="https://colab.research.google.com/github/yinhao0424/reuster/blob/master/ReusterFewShotLearner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference:
- Paper/Blog
  - [BERT word embedding](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/#31-running-bert-on-our-text)
  - [triplet-network-pytorch](https://github.com/andreasveit/triplet-network-pytorch/blob/master/train.py)
  - [Sentence Embeddings using Siamese BERT-Networks - paper](https://www.aclweb.org/anthology/D19-1410.pdf)
  - [Sentence Embeddings using Siamese BERT-Networks - colab](https://github.com/aneesha/SiameseBERT-Notebook/blob/master/SiameseBERT_SemanticSearch.ipynb)
- Disscussion
    - [Generate sequence classifier](https://github.com/huggingface/transformers/issues/1001)


In [1]:
# a specific version of transformaer has been used 
! pip install -q transformers==3.0.2
# !pip install -q transformers

     |████████████████████████████████| 778kB 4.2MB/s 
     |████████████████████████████████| 1.1MB 53.3MB/s 
     |████████████████████████████████| 890kB 52.1MB/s 
     |████████████████████████████████| 3.0MB 50.9MB/s 


In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel

import warnings
warnings.simplefilter('ignore')
import logging
logging.basicConfig(level=logging.ERROR)

In [3]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
device

device(type='cpu')

In [4]:
# Sections of config
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 2
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [5]:
reuster_train = pd.read_csv('/content/drive/MyDrive/data/reuters/reuster_fewshot_train.csv')
reuster_train.head()

,id,topics,texts
0,4016,iron-steel,"usx <x> proved oil, gas reserves fall in 1986u..."
1,4022,carcass,argentine meat exports higher in jan/feb 1987a...
2,4022,livestock,argentine meat exports higher in jan/feb 1987a...
3,4035,veg-oil,british minister criticises proposed ec oils t...
4,4040,oilseed,china's rapeseed crop damaged by stormsthe yie...


In [6]:
reuster_train.shape

(1143, 3)

In [7]:
# #iron-steel
# reuster_train.topics[0]
# topics = reuster_train.topics
# candi = topics[topics!='iron-steel'].index
# idx = np.random.choice(candi)

# print(topics[idx])

In [8]:
class FewShotDataset(Dataset):
    """
        Input: a dataframe
        output: anchor, positive and negative
    """
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.texts
        self.topics = self.data.topics
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        anchor = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )

        ids = anchor['input_ids']
        mask = anchor['attention_mask']
        # token_type_ids = anchor["token_type_ids"]


        return {'anchor':{
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)},
        'positive': self.get_positive(index),
        'negative': self.get_negative(index)
        }

    def get_positive(self, index):
         # the topic
        topic = self.topics[index]

        # select positive data which have the same topic with the anchor
        candidates = self.topics[self.topics==topic].index
        p_idx = index
        while p_idx == index:
          p_idx = np.random.choice(candidates)
        
        text = str(self.text[p_idx])
        text = " ".join(text.split())

        positive = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )

        ids = positive['input_ids']
        mask = positive['attention_mask']
        # token_type_ids = positive["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)}

    def get_negative(self, index):
         # the topic
        topic = self.topics[index]

        # select positive data which have the same topic with the anchor
        candidates = self.topics[self.topics!=topic].index
        n_idx = index
        n_idx = np.random.choice(candidates)
        
        text = str(self.text[n_idx])
        text = " ".join(text.split())

        negative = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = negative['input_ids']
        mask = negative['attention_mask']
        # token_type_ids = negative["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)}

In [9]:
print("TRAIN Dataset: {}".format(reuster_train.shape))
# print("TEST Dataset: {}".format(test_data.shape))

training_set = FewShotDataset(reuster_train, tokenizer, MAX_LEN)
# testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

TRAIN Dataset: (1143, 3)


In [10]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
# testing_loader = DataLoader(testing_set, **test_params)

In [11]:
print("The len of training loader is {}.".format(len(training_loader)))

The len of training loader is 143.


## Create the Neural Network for Fine Tuning

In [12]:
class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 256)

    def forward(self, data):
        input_ids = data['ids']
        attention_mask = data['mask']

        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

    # def forward(self, anchor,positive,negative):
    #     res_anchor = self.forward_once(anchor)
    #     res_positive = self.forward_once(positive)
    #     res_negative = self.forward_once(negative)
    #     return res_anchor,res_positive,res_negative

model = DistilBERTClass()
model.to(device)

DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_featu

In [13]:
def triplet_loss(anchor, positive, negative):
  loss = torch.nn.TripletMarginLoss(margin=1.0, p=2)
  return loss(anchor, positive, negative)

In [14]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [15]:
# model input: input_ids, attention_mask, token_type_ids
# training_loader:   return {'anchor':{
        #     'ids': torch.tensor(ids, dtype=torch.long),
        #     'mask': torch.tensor(mask, dtype=torch.long),
        #     'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)},
        # 'positive': self.get_positive(index),
        # 'negative': self.get_negative(index)
        # }
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        anchor = data['anchor']
        positive = data['positive']
        negative = data['negative']
        res_anchor,res_positive,res_negative = model(anchor),model(positive),model(negative)
        # res_anchor,res_positive,res_negative = model(anchor,positive,negative)

        optimizer.zero_grad()
        loss = triplet_loss(res_anchor,res_positive,res_negative)
        if _%20==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [16]:
# 101,  7592,  1010,  2026,  3899,  2003, 10140,   102]
# for _,data in tqdm(enumerate(training_loader, 0)):
#     anchor = data['anchor']
#     positive = data['positive']
#     negative = data['negative']
#     # print(anchor)
#     res_anchor,res_positive,res_negative = model(anchor),model(positive),model(negative)
#     print(res_anchor)
#     break


0it [00:00, ?it/s]

tensor([[ 0.0150,  0.2781,  0.0539,  ..., -0.0671, -0.1414, -0.2027],
        [-0.0738,  0.2706,  0.0399,  ..., -0.1368, -0.1213, -0.2575],
        [-0.0792,  0.2642,  0.0393,  ..., -0.0070, -0.1887, -0.1760],
        ...,
        [-0.0014,  0.2907,  0.0887,  ..., -0.1080,  0.0004, -0.1858],
        [ 0.0423,  0.1955, -0.0205,  ..., -0.0482, -0.1494, -0.1603],
        [ 0.1207,  0.2113,  0.0879,  ..., -0.0489, -0.0202, -0.0949]],
       grad_fn=<AddmmBackward>)


In [17]:

for epoch in range(EPOCHS):
    train(epoch)


0it [00:00, ?it/s]

Epoch: 0, Loss:  0.9176938533782959



1it [00:30, 30.93s/it]
2it [01:00, 30.60s/it]
3it [01:30, 30.26s/it]
4it [01:59, 30.01s/it]
5it [02:29, 29.87s/it]
6it [02:59, 30.00s/it]
7it [03:29, 29.87s/it]
8it [03:58, 29.75s/it]
9it [04:28, 29.67s/it]
10it [04:57, 29.57s/it]
11it [05:26, 29.51s/it]
12it [05:56, 29.55s/it]
13it [06:25, 29.49s/it]
14it [06:55, 29.48s/it]
15it [07:24, 29.54s/it]
16it [07:54, 29.57s/it]
17it [08:24, 29.55s/it]
18it [08:53, 29.58s/it]
19it [09:23, 29.60s/it]
20it [09:52, 29.59s/it]

Epoch: 0, Loss:  0.9406020045280457



21it [10:22, 29.51s/it]
22it [10:51, 29.47s/it]
23it [11:21, 29.48s/it]
24it [11:50, 29.51s/it]
25it [12:20, 29.55s/it]
26it [12:49, 29.58s/it]
27it [13:20, 29.78s/it]
28it [13:49, 29.78s/it]
29it [14:19, 29.72s/it]
30it [14:49, 29.74s/it]
31it [15:19, 29.73s/it]
32it [15:48, 29.74s/it]
33it [16:18, 29.66s/it]
34it [16:47, 29.58s/it]
35it [17:17, 29.52s/it]
36it [17:46, 29.60s/it]
37it [18:16, 29.63s/it]
38it [18:46, 29.62s/it]
39it [19:15, 29.59s/it]
40it [19:45, 29.63s/it]

...


101it [49:52, 29.65s/it]
102it [50:21, 29.62s/it]
103it [50:51, 29.60s/it]
104it [51:20, 29.58s/it]
105it [51:50, 29.53s/it]
106it [52:19, 29.46s/it]
107it [52:49, 29.46s/it]
108it [53:18, 29.45s/it]
109it [53:47, 29.39s/it]
110it [54:17, 29.35s/it]
111it [54:47, 29.58s/it]
112it [55:16, 29.56s/it]
113it [55:45, 29.42s/it]
114it [56:14, 29.35s/it]
115it [56:44, 29.30s/it]
116it [57:13, 29.29s/it]
117it [57:42, 29.27s/it]
118it [58:11, 29.21s/it]
119it [58:40, 29.20s/it]
120it [59:10, 29.19s/it]

Epoch: 1, Loss:  0.5366882085800171



121it [59:39, 29.21s/it]
122it [1:00:08, 29.31s/it]
123it [1:00:38, 29.30s/it]
124it [1:01:07, 29.34s/it]
125it [1:01:36, 29.26s/it]
126it [1:02:05, 29.18s/it]
127it [1:02:34, 29.14s/it]
128it [1:03:03, 29.14s/it]
129it [1:03:32, 29.15s/it]
130it [1:04:02, 29.16s/it]
131it [1:04:31, 29.16s/it]
132it [1:05:01, 29.42s/it]
133it [1:05:30, 29.37s/it]
134it [1:05:59, 29.36s/it]
135it [1:06:29, 29.32s/it]
136it [1:06:58, 29.29s/it]
137it [1:07:27, 29.28s/it]
138it [1:07:56, 29.23s/it]
139it [1:08:25, 29.21s/it]
140it [1:08:55, 29.19s/it]

Epoch: 1, Loss:  0.10475233197212219



141it [1:09:24, 29.28s/it]
142it [1:09:53, 29.32s/it]
143it [1:10:19, 29.51s/it]
